In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB

In [5]:
loan = pd.read_csv("loan.csv",index_col = 0)

In [7]:
loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [11]:
loan.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, LP001002 to LP002990
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 62.4+ KB


In [125]:
X = loan.drop('Loan_Status',axis = 1)
y = loan[['Loan_Status']]

In [127]:
imput_simple_num = SimpleImputer(strategy = 'median').set_output(transform='pandas')
imput_simple_obj = SimpleImputer(strategy = 'constant',  fill_value = 'unknown').set_output(transform='pandas')


In [130]:
ct = make_column_transformer((imput_simple_num,make_column_selector(dtype_exclude = object) ),
                                  (imput_simple_obj,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')


In [132]:
ct = ct.set_output(transform='pandas')

In [134]:
# X_imp = ct.fit_transform(X)
# y_imp = ct.fit_transform(y)

In [97]:
#loan_df = loan_df.set_output(transform='pandas')

In [99]:
loan_df.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender', 'Married', 'Dependents',
       'Education', 'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [136]:
X_imp.isnull().sum()

ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
Property_Area        0
dtype: int64

In [138]:
y_imp.isnull().sum()

Loan_Status    0
dtype: int64

In [109]:
ohe = OneHotEncoder(sparse_output = False,handle_unknown = 'ignore').set_output(transform='pandas')


In [111]:
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')


In [142]:
# X_imp_ohe = ct_ohe.fit_transform(X_imp)
# y_imp_ohe = ct_ohe.fit_transform(y_imp)

In [146]:
X_imp_ohe.dtypes


ApplicantIncome            float64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History             float64
Gender_Female              float64
Gender_Male                float64
Gender_unknown             float64
Married_No                 float64
Married_Yes                float64
Married_unknown            float64
Dependents_0               float64
Dependents_1               float64
Dependents_2               float64
Dependents_3+              float64
Dependents_unknown         float64
Education_Graduate         float64
Education_Not Graduate     float64
Self_Employed_No           float64
Self_Employed_Yes          float64
Self_Employed_unknown      float64
Property_Area_Rural        float64
Property_Area_Semiurban    float64
Property_Area_Urban        float64
dtype: object

In [234]:
lr = LogisticRegression(random_state = 24)
pipe = Pipeline([('IMP',ct),('OHE',ct_ohe),('LR',lr)])
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
params = {'LR__solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],
         'LR__C': np.linspace(0.001,1,11)}

In [236]:
gcv = GridSearchCV(pipe, param_grid = params,scoring = 'roc_auc', cv = kfold, verbose = 3)
gcv.fit(X,y)

Fitting 5 folds for each of 66 candidates, totalling 330 fits
[CV 1/5] END .....LR__C=0.001, LR__solver=lbfgs;, score=0.442 total time=   0.0s
[CV 2/5] END .....LR__C=0.001, LR__solver=lbfgs;, score=0.515 total time=   0.0s
[CV 3/5] END .....LR__C=0.001, LR__solver=lbfgs;, score=0.448 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was ex

[CV 4/5] END .....LR__C=0.001, LR__solver=lbfgs;, score=0.635 total time=   0.0s
[CV 5/5] END .....LR__C=0.001, LR__solver=lbfgs;, score=0.565 total time=   0.0s
[CV 1/5] END .LR__C=0.001, LR__solver=liblinear;, score=0.438 total time=   0.0s
[CV 2/5] END .LR__C=0.001, LR__solver=liblinear;, score=0.633 total time=   0.0s
[CV 3/5] END .LR__C=0.001, LR__solver=liblinear;, score=0.470 total time=   0.0s
[CV 4/5] END .LR__C=0.001, LR__solver=liblinear;, score=0.632 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was ex

[CV 5/5] END .LR__C=0.001, LR__solver=liblinear;, score=0.567 total time=   0.0s
[CV 1/5] END .LR__C=0.001, LR__solver=newton-cg;, score=0.487 total time=   0.0s
[CV 2/5] END .LR__C=0.001, LR__solver=newton-cg;, score=0.522 total time=   0.0s


...

[CV 1/5] END .........LR__C=1.0, LR__solver=sag;, score=0.424 total time=   0.0s
[CV 2/5] END .........LR__C=1.0, LR__solver=sag;, score=0.549 total time=   0.0s
[CV 3/5] END .........LR__C=1.0, LR__solver=sag;, score=0.466 total time=   0.0s
[CV 4/5] END .........LR__C=1.0, LR__solver=sag;, score=0.530 total time=   0.0s
[CV 5/5] END .........LR__C=1.0, LR__solver=sag;, score=0.448 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENT

[CV 1/5] END ........LR__C=1.0, LR__solver=saga;, score=0.436 total time=   0.0s
[CV 2/5] END ........LR__C=1.0, LR__solver=saga;, score=0.539 total time=   0.0s
[CV 3/5] END ........LR__C=1.0, LR__solver=saga;, score=0.485 total time=   0.0s
[CV 4/5] END ........LR__C=1.0, LR__solver=saga;, score=0.542 total time=   0.0s
[CV 5/5] END ........LR__C=1.0, LR__solver=saga;, score=0.466 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENT

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('IMP',
                                        ColumnTransformer(transformers=[('simpleimputer-1',
                                                                         SimpleImputer(strategy='median'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002E36A0C63F0>),
                                                                        ('simpleimputer-2',
                                                                         SimpleImputer(fill_value='unknown',
                                                                                       strategy='constant')...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002E367B8F4D0>)],
                                                          verbose_feature_names_out=False)),
                                       ('LR',
                                        LogisticRegression(random_state=24))]),
             param_grid={'LR__C': array([0.001 , 0.1009, 0.2008, 0.3007, 0.4006, 0.5005, 0.6004, 0.7003,
       0.8002, 0.9001, 1.    ]),
                         'LR__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']},
             scoring='roc_auc', verbose=3)

In [237]:
print("Best Param:",gcv.best_params_)
print("Best Score:",gcv.best_score_)

Best Param: {'LR__C': 0.1009, 'LR__solver': 'liblinear'}
Best Score: 0.7618696439439473


In [238]:
# inferencing 

In [239]:
bm = gcv.best_estimator_

In [241]:
test = pd.read_csv('test.csv')
# sample['Loan_Status'] = y_pred
# sample.to_csv('knn_5_nov_2.csv',index=False)

In [256]:
y_pred = bm.predict(test)

In [258]:
bm.predict(test)

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [269]:
y_pred

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [271]:
submit = pd.read_csv("sample_submission.csv")
submit['Loan_Status'] = y_pred
submit.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y


In [273]:
# tHIS 'submit'  data can be exported to .csv 

In [316]:
submit.to_csv('Loan_6Nov_LR_1.csv',index=False)

In [322]:
# KneighborClassification

In [324]:
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()

In [338]:
knc = KNeighborsClassifier()
scl_mm, scl_std = MinMaxScaler(),StandardScaler()
pipe = Pipeline([('IMP',ct),('OHE',ct_ohe),('SCL',None),('KNC',knc)])
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
params = {'KNC__n_neighbors': np.arange(1, 11),
          'SCL': [None, scl_mm,scl_std],
          'KNC__weights': ['uniform', 'distance'],  # weight function for neighbors
            'KNC__metric': ['euclidean', 'manhattan'] 
         }


gcv = GridSearchCV(pipe,param_grid = params,scoring = 'roc_auc',verbose = 3, cv = kfold) 
gcv.fit(X,y)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=None;, score=0.512 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV 2/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=None;, score=0.512 total time=   0.0s
[CV 3/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=None;, score=0.484 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV 4/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=None;, score=0.467 total time=   0.0s
[CV 5/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=None;, score=0.439 total time=   0.0s
[CV 1/5] END KNC__metric=euclidean, KNC__n_neighbors=1, KNC__weights=uniform, SCL=MinMaxScaler();, score=0.630 total time=   0.0s


...

[CV 3/5] END KNC__metric=manhattan, KNC__n_neighbors=10, KNC__weights=distance, SCL=StandardScaler();, score=0.616 total time=   0.0s
[CV 4/5] END KNC__metric=manhattan, KNC__n_neighbors=10, KNC__weights=distance, SCL=StandardScaler();, score=0.752 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV 5/5] END KNC__metric=manhattan, KNC__n_neighbors=10, KNC__weights=distance, SCL=StandardScaler();, score=0.752 total time=   0.0s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('IMP',
                                        ColumnTransformer(transformers=[('simpleimputer-1',
                                                                         SimpleImputer(strategy='median'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002E36A0C63F0>),
                                                                        ('simpleimputer-2',
                                                                         SimpleImputer(fill_value='unknown',
                                                                                       strategy='constant')...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002E367B8F4D0>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', None),
                                       ('KNC', KNeighborsClassifier())]),
             param_grid={'KNC__metric': ['euclidean', 'manhattan'],
                         'KNC__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'KNC__weights': ['uniform', 'distance'],
                         'SCL': [None, MinMaxScaler(), StandardScaler()]},
             scoring='roc_auc', verbose=3)

In [340]:
print("Best Param:",gcv.best_params_)
print("Best Score:",gcv.best_score_)

Best Param: {'KNC__metric': 'euclidean', 'KNC__n_neighbors': 5, 'KNC__weights': 'distance', 'SCL': MinMaxScaler()}
Best Score: 0.7263404363094766


In [342]:
bm = gcv.best_estimator_

In [344]:
test = pd.read_csv('test.csv')

In [346]:
y_pred = bm.predict(test)
bm.predict(test)

array(['Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'N', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [348]:
submit = pd.read_csv("sample_submission.csv")
submit['Loan_Status'] = y_pred
submit.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,N


In [350]:
submit.to_csv('Loan_6Nov_KNC_1.csv',index=False)